In [1]:
#importing libs
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedShuffleSplit
from sklearn.metrics import confusion_matrix
#import sweetviz as sv
#import shap
#import dask
#from dask_ml.model_selection import GridSearchCV as GS
#from array import array

In [2]:
#Settings
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

sns.set_style('darkgrid')
sns.set(rc={'figure.figsize':(15,8)})

In [3]:
df_members = pd.read_csv('../input/wsdmmusic/members.csv')
df_songs = pd.read_csv('../input/wsdmmusic/songs.csv')
df_train = pd.read_csv('../input/wsdmmusic/train.csv')
df_test = pd.read_csv('../input/wsdmmusic/test.csv')
df_songs_info = pd.read_csv('../input/wsdmmusic/song_extra_info.csv')

In [4]:
df_members.head()

,msno,city,bd,gender,registered_via,registration_init_time,expiration_date
0,XQxgAYj3klVKjR3oxPPXYYFp4soD4TuBghkhMTD4oTw=,1,0,NaN,7,20110820,20170920
1,UizsfmJb9mV54qE9hCYyU07Va97c0lCRLEQX3ae+ztM=,1,0,NaN,7,20150628,20170622
2,D8nEhsIOBSoE6VthTaqDX8U6lqjJ7dLdr72mOyLya2A=,1,0,NaN,4,20160411,20170712
3,mCuD+tZ1hERA/o5GPqk38e041J8ZsBaLcu7nGoIIvhI=,1,0,NaN,9,20150906,20150907
4,q4HRBfVSssAFS9iRfxWrohxuk9kCYMKjHOEagUMV6rQ=,1,0,NaN,4,20170126,20170613


In [5]:
df_songs.head()

,song_id,song_length,genre_ids,artist_name,composer,lyricist,language
0,CXoTN1eb7AI+DntdU1vbcwGRV4SCIDxZu+YD8JP8r4E=,247640,465,張信哲 (Jeff Chang),董貞,何啟弘,3.0
1,o0kFgae9QtnYgRkVPqLJwa05zIhRlUjfF7O1tDw0ZDU=,197328,444,BLACKPINK,TEDDY| FUTURE BOUNCE| Bekuh BOOM,TEDDY,31.0
2,DwVvVurfpuz+XPuFvucclVQEyPqcpUkHR0ne1RQzPs0=,231781,465,SUPER JUNIOR,NaN,NaN,31.0
3,dKMBWoZyScdxSkihKG+Vf47nc18N9q4m58+b4e7dSSE=,273554,465,S.H.E,湯小康,徐世珍,3.0
4,W3bqWd3T+VeHFzHAUfARgW9AvVRaF4N5Yzm4Mr6Eo/o=,140329,726,貴族精選,Traditional,Traditional,52.0


In [6]:
df_train.head()

,msno,song_id,source_system_tab,source_screen_name,source_type,target
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,my library,Local playlist more,local-playlist,1
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,explore,Explore,online-playlist,1


In [7]:
df_test.head()

,id,msno,song_id,source_system_tab,source_screen_name,source_type
0,0,V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=,WmHKgKMlp1lQMecNdNvDMkvIycZYHnFwDT72I5sIssc=,my library,Local playlist more,local-library
1,1,V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=,y/rsZ9DC7FwK5F2PK2D5mj+aOBUJAjuu3dZ14NgE0vM=,my library,Local playlist more,local-library
2,2,/uQAlrAkaczV+nWCd2sPF2ekvXPRipV7q0l+gbLuxjw=,8eZLFOdGVdXBSqoAv5nsLigeH2BvKXzTQYtUM53I0k4=,discover,NaN,song-based-playlist
3,3,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,ztCf8thYsS4YN3GcIL/bvoxLm/T5mYBVKOO4C9NiVfQ=,radio,Radio,radio
4,4,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,MKVMpslKcQhMaFEgcEQhEfi5+RZhMYlU3eRDpySrH8Y=,radio,Radio,radio


In [8]:
df_songs_info.head()

,song_id,name,isrc
0,LP7pLJoJFBvyuUwvu+oLzjT+bI+UeBPURCecJsX1jjs=,我們,TWUM71200043
1,ClazTFnk6r0Bnuie44bocdNMM3rdlrq0bCGAsGUWcHE=,Let Me Love You,QMZSY1600015
2,u2ja/bZE3zhCGxvbbOB3zOoUjx27u40cf5g09UXMoKQ=,原諒我,TWA530887303
3,92Fqsy0+p6+RHe2EoLKjHahORHR1Kq1TBJoClW9v+Ts=,Classic,USSM11301446
4,0QFmz/+rJy1Q56C1DuYqT9hKKqi5TUqx0sN0IwvoHrw=,愛投羅網,TWA471306001


We will merge columns from other dataframes into train and test.

In [9]:
df_train = pd.merge(df_train,df_songs,how='left',on=['song_id'])
df_test = pd.merge(df_test,df_songs,how='left',on=['song_id'])
df_train = pd.merge(df_train,df_members,how='left',on=['msno'])
df_test = pd.merge(df_test,df_members,how='left',on=['msno'])
df_train = pd.merge(df_train,df_songs_info,how='left',on=['song_id'])
df_test = pd.merge(df_test,df_songs_info,how='left',on=['song_id'])

In [10]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7377418 entries, 0 to 7377417
Data columns (total 20 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   msno                    object 
 1   song_id                 object 
 2   source_system_tab       object 
 3   source_screen_name      object 
 4   source_type             object 
 5   target                  int64  
 6   song_length             float64
 7   genre_ids               object 
 8   artist_name             object 
 9   composer                object 
 10  lyricist                object 
 11  language                float64
 12  city                    int64  
 13  bd                      int64  
 14  gender                  object 
 15  registered_via          int64  
 16  registration_init_time  int64  
 17  expiration_date         int64  
 18  name                    object 
 19  isrc                    object 
dtypes: float64(2), int64(6), object(12)
memory usage: 1.2+ GB


In [11]:
df_train.isnull().sum()

msno                            0
song_id                         0
source_system_tab           24849
source_screen_name         414804
source_type                 21539
target                          0
song_length                   114
genre_ids                  118455
artist_name                   114
composer                  1675706
lyricist                  3178798
language                      150
city                            0
bd                              0
gender                    2961479
registered_via                  0
registration_init_time          0
expiration_date                 0
name                         1457
isrc                       577858
dtype: int64

In [12]:
for col in df_train.select_dtypes(include=['object']).columns:
    df_train[col] = df_train[col].fillna('Unknown')

In [13]:
for col in df_train.select_dtypes(include=['object']).columns:
    df_train[col] = df_train[col].astype('category')

In [14]:
df_train.isnull().sum()

msno                        0
song_id                     0
source_system_tab           0
source_screen_name          0
source_type                 0
target                      0
song_length               114
genre_ids                   0
artist_name                 0
composer                    0
lyricist                    0
language                  150
city                        0
bd                          0
gender                      0
registered_via              0
registration_init_time      0
expiration_date             0
name                        0
isrc                        0
dtype: int64

We have negligiable amount of NA coulmns which we can drop

In [15]:
df_train.dropna(inplace=True)

Changing date columns to datetime format

In [16]:
df_train['registration_init_time'] = pd.to_datetime(df_train['registration_init_time'],format='%Y%m%d')
df_train['expiration_date'] = pd.to_datetime(df_train['expiration_date'],format='%Y%m%d')

We will create a new feature called mebership duration that will be the time delta between expiration date and registration date.The value will be in months with 2 decimal values.

In [17]:
df_train['membership_duration'] = round((df_train['expiration_date'] - df_train['registration_init_time'])/np.timedelta64(1,'M'),2)

Seems as though some values of expiration date is less than registration date. Possibily the data was entered incorrectly. So we will drop those rows.

In [18]:
df_train.drop(df_train[df_train['membership_duration']<0].index,inplace=True)

In [19]:
#Changing city, language, registered_via and target columns to category
df_train['city'] = df_train['city'].astype('category')
df_train['target'] = df_train['target'].astype('category')
df_train['language'] = df_train['language'].astype('category')
df_train['registered_via'] = df_train['registered_via'].astype('category')

Sum of unique values under each column

In [20]:
for column in df_train.select_dtypes(include=['category']).columns:
    unique = len(pd.unique(df_train[column]))
    print(f'No of unique values in {column} : {unique}')

No of unique values in msno : 30754
No of unique values in song_id : 359913
No of unique values in source_system_tab : 9
No of unique values in source_screen_name : 20
No of unique values in source_type : 13
No of unique values in target : 2
No of unique values in genre_ids : 573
No of unique values in artist_name : 40582
No of unique values in composer : 76063
No of unique values in lyricist : 33887
No of unique values in language : 10
No of unique values in city : 21
No of unique values in gender : 3
No of unique values in registered_via : 5
No of unique values in name : 234111
No of unique values in isrc : 269720


Basic analysis and encoding categorical features. We will use the LabelEncoder.

In [21]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7377265 entries, 0 to 7377417
Data columns (total 21 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   msno                    category      
 1   song_id                 category      
 2   source_system_tab       category      
 3   source_screen_name      category      
 4   source_type             category      
 5   target                  category      
 6   song_length             float64       
 7   genre_ids               category      
 8   artist_name             category      
 9   composer                category      
 10  lyricist                category      
 11  language                category      
 12  city                    category      
 13  bd                      int64         
 14  gender                  category      
 15  registered_via          category      
 16  registration_init_time  datetime64[ns]
 17  expiration_date         datetime64[ns]
 18  na

In [22]:
df_train['msno'] = LabelEncoder().fit_transform(df_train['msno'])
df_train['song_id'] = LabelEncoder().fit_transform(df_train['song_id'])
df_train['source_system_tab'] = LabelEncoder().fit_transform(df_train['source_system_tab'])
df_train['source_screen_name'] = LabelEncoder().fit_transform(df_train['source_screen_name'])
df_train['source_type'] = LabelEncoder().fit_transform(df_train['source_type'])
df_train['artist_name'] = LabelEncoder().fit_transform(df_train['artist_name'])
df_train['gender'] = LabelEncoder().fit_transform(df_train['gender'])
df_train['name'] = LabelEncoder().fit_transform(df_train['name'])
df_train['composer'] = LabelEncoder().fit_transform(df_train['composer'])
df_train['registered_via'] = LabelEncoder().fit_transform(df_train['registered_via'])
df_train['lyricist'] = LabelEncoder().fit_transform(df_train['lyricist'])
df_train['genre_ids'] = LabelEncoder().fit_transform(df_train['genre_ids'])

In [23]:
#Some category vals getting assigned as int64 data type. We will chnage those back to category
for col in df_train.select_dtypes(include=['int64']).columns:
    df_train[col] = df_train[col].astype('category')

In [24]:
df_train['registered_via'] = df_train['registered_via'].astype('category')
df_train['bd'] = df_train['bd'].astype('int64')

In [25]:
for column in df_train.select_dtypes(include=['category']).columns:
    unique = len(pd.unique(df_train[column]))
    print(f'No of unique values in {column} : {unique}')

No of unique values in msno : 30754
No of unique values in song_id : 359913
No of unique values in source_system_tab : 9
No of unique values in source_screen_name : 20
No of unique values in source_type : 13
No of unique values in target : 2
No of unique values in genre_ids : 573
No of unique values in artist_name : 40582
No of unique values in composer : 76063
No of unique values in lyricist : 33887
No of unique values in language : 10
No of unique values in city : 21
No of unique values in gender : 3
No of unique values in registered_via : 5
No of unique values in name : 234111
No of unique values in isrc : 269720


In [26]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7377265 entries, 0 to 7377417
Data columns (total 21 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   msno                    category      
 1   song_id                 category      
 2   source_system_tab       category      
 3   source_screen_name      category      
 4   source_type             category      
 5   target                  category      
 6   song_length             float64       
 7   genre_ids               category      
 8   artist_name             category      
 9   composer                category      
 10  lyricist                category      
 11  language                category      
 12  city                    category      
 13  bd                      int64         
 14  gender                  category      
 15  registered_via          category      
 16  registration_init_time  datetime64[ns]
 17  expiration_date         datetime64[ns]
 18  na

We could see no of unique columns before encoding mening all unique values have been covered.

We will drop date columns as well as the isrc column as its mentioed not to be acurate

In [27]:
df_train.drop(columns=['registration_init_time','expiration_date','isrc'],inplace=True)

In [28]:
#df_train['target'] = df_train['target'].astype('bool')
#changing song length to minutes
df_train['song_length'] = df_train['song_length']/60000

In [29]:
X_train, X_test, y_train, y_test = train_test_split(df_train[df_train.columns[df_train.columns!='target']],df_train.target,test_size=0.30)

In [30]:
#Using sweetviz to get an analysis on features relation with target variable
#sample = df_train.head(10000)
#eda = sv.analyze(sample,'target')
#eda.show_html('eda_analysis')

NameError: name 'sv' is not defined

In [59]:
sample = df_train.head(500000)
X_train, X_test, y_train, y_test = train_test_split(sample[sample.columns[sample.columns!='target']],sample.target,test_size=0.30)

In [60]:
params = [{'n_estimators':[50,100,150],'max_depth':[25,50,75],'max_features' : ['auto','sqrt',None],'bootstrap':[True],'oob_score':[True]}]
cv_set = GridSearchCV(estimator = RandomForestClassifier(),param_grid=params,scoring='roc_auc',n_jobs=-1,refit=True,return_train_score=True,verbose=2)

In [ ]:
cv_result = cv_set.fit(X_train,y_train)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


In [34]:
cv_result.best_params_

{'bootstrap': True,
 'max_depth': 75,
 'max_features': None,
 'n_estimators': 150,
 'oob_score': True}

In [35]:
cv_result.best_score_

0.8678762430575089

In [32]:
#X_train, X_test, y_train, y_test = train_test_split(df_train[df_train.columns[df_train.columns!='target']],df_train.target,test_size=0.30)

In [ ]:
model = RandomForestClassifier(n_estimators=150,max_depth=75,verbose=20,n_jobs=4,bootstrap=True,oob_score=True,warm_start=True)
model.fit(X_train,y_train)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 1 of 150building tree 2 of 150
building tree 3 of 150
building tree 4 of 150



[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:   53.6s


building tree 5 of 150


[Parallel(n_jobs=4)]: Done   2 tasks      | elapsed:   54.3s
[Parallel(n_jobs=4)]: Done   3 tasks      | elapsed:   54.4s


...

building tree 57 of 150


[Parallel(n_jobs=4)]: Done  54 tasks      | elapsed: 12.7min


building tree 58 of 150


[Parallel(n_jobs=4)]: Done  55 tasks      | elapsed: 12.8min


building tree 59 of 150


In [ ]:
y_pred = model.predict(X_test)

In [ ]:
confusion_matrix(y_test,y_pred,normalize='true')